In [1]:
import torch
# import torchvision
import torchtext
from torch import nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from perceiver_pytorch import Perceiver
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import itertools
from torch.utils.tensorboard import SummaryWriter
import os
from tqdm.notebook import tqdm
from datetime import datetime
from babi.babi_loader import BabiDataset, pad_collate

In [6]:
import torch
from perceiver_pytorch import PerceiverIO

model = PerceiverIO(
    dim = 13,                    # dimension of sequence to be encoded
    queries_dim = 32,            # dimension of decoder queries
    logits_dim = 50,            # dimension of final logits
    depth = 6,                   # depth of net
    num_latents = 256,           # number of latents, or induced set points, or centroids. different papers giving it different names
    latent_dim = 512,            # latent dimension
    cross_heads = 1,             # number of heads for cross attention. paper said 1
    latent_heads = 8,            # number of heads for latent self attention, 8
    cross_dim_head = 64,         # number of dimensions per cross attention head
    latent_dim_head = 64,        # number of dimensions per latent self attention head
    weight_tie_layers = False,   # whether to weight tie layers (optional, as indicated in the diagram)
    self_per_cross_attn = 2,     # number of self attention blocks per cross attention
)

# seq = torch.randn(1, 512, 32)
# queries = torch.randn(1, 128, 32)

# logits = model(seq, queries = queries) # (1, 128, 100) - (batch, decoder seq, logits dim)

# add bAbI query as latent query
# only one query/question and answer per forward pass

seq = torch.randn(1, 70, 13) # (batch, input_sequence_length, input_sequence_dim/dim)
queries = torch.randn(1, 1, 32) # (batch, output_sequence_length, queries_dim) - probably should be learned

logits = model(seq, queries = queries) # (batch, output_sequence_length, output_sequence_dim/logits dim)
print(logits.shape)

torch.Size([1, 1, 50])


In [3]:
batch_size=128
max_epochs = 2

max_con_num_sentences, max_con_num_words, max_qus_num_words = 0, 0, 0
ans_lis = []
# c=0
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda:0" if use_cuda else "cpu")
# print(device)
# torch.backends.cudnn.benchmark = True

# model.to(device)
# criterion = nn.CrossEntropyLoss(size_average=False)

for task_id in range(1, 21):
    dset = BabiDataset(task_id, 
                       ds_path='/home/gabriel/Documents/datasets/bAbi/en/qa{}_*', 
                       vocab_path='/home/gabriel/Documents/datasets/bAbi/en/babi{}_vocab.pkl')
    vocab_size = len(dset.QA.VOCAB)
    dset.set_mode('train')
    train_loader = DataLoader(
        dset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate
    )
    
#     c += len(train_loader.dataset)
    
    for batch_idx, data in enumerate(train_loader):
#         optimizer.zero_grad()
        contexts, questions, answers = data
        contexts = Variable(contexts.long().cuda())
        questions = Variable(questions.long().cuda())
        answers = Variable(answers.cuda())
#         print(contexts.shape, questions.shape, answers.shape)
        _, con_num_sentences, con_num_words = contexts.shape
        _, qus_num_words = questions.shape
        if con_num_sentences > max_con_num_sentences:
            max_con_num_sentences = con_num_sentences
        if con_num_words > max_con_num_words:
            max_con_num_words = con_num_words
        if qus_num_words > max_qus_num_words:
            max_qus_num_words = qus_num_words
            
        for ans in answers:
            ans_lis.append(int(ans))
        
#     dset.set_mode('valid')
#     valid_loader = DataLoader(
#         dset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate
#     )
     
    dset.set_mode('test')
    train_loader = DataLoader(
        dset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate
    )
    
    for batch_idx, data in enumerate(train_loader):
        contexts, questions, answers = data
        contexts = Variable(contexts.long().cuda())
        questions = Variable(questions.long().cuda())
        answers = Variable(answers.cuda())
        for ans in answers:
            ans_lis.append(int(ans))
            
    dset.set_mode('valid')
    train_loader = DataLoader(
        dset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate
    )
    
    for batch_idx, data in enumerate(train_loader):
        contexts, questions, answers = data
        contexts = Variable(contexts.long().cuda())
        questions = Variable(questions.long().cuda())
        answers = Variable(answers.cuda())
        for ans in answers:
            ans_lis.append(int(ans))
        
ans_lis = set(ans_lis)
print(max_con_num_sentences, max_con_num_words, max_qus_num_words)
print(ans_lis)
# print(c)

70 13 13
{2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49}


In [4]:




# # checkpoint = torch.load(f'checkpoints/{model_name}/best_checkpoint.pt')
# # model.load_state_dict(checkpoint['state_dict'])

# # criterion = nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# val_loss_min = 100000.0

# writer = SummaryWriter()
# if not os.path.isdir(f'checkpoints/{model_name}'):
#     os.mkdir(f'checkpoints/{model_name}')

# now = datetime.now().strftime("%d_%m_%Y__%H_%M_%S")
    
# for epoch in range(max_epochs):
#     train_loss = 0.0
#     val_loss = 0.0
    
#     model.train()
#     loss_sum = 0
#     loss_count = 0
    
#     for batch_idx, (X_batch, y_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch + 1}/{max_epochs}")):
#         optimizer.zero_grad()
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         X_batch = torch.transpose(torch.transpose(X_batch, 1, 2), 2, 3)
# #         from matplotlib import pyplot as plt
# #         plt.imshow(X_batch[0].cpu().numpy())
# #         plt.show()
# #         print(X_batch.shape, y_batch.shape)
#         y_pred = model(X_batch)
#         loss = criterion(y_pred, y_batch)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.data
#         writer.add_scalars("losses_step", {"train_loss": loss.data}, epoch * len(train_loader) + batch_idx)
        
#     model.eval()
#     with torch.no_grad():
#         for batch_idx, (X_batch, y_batch) in enumerate(val_loader):
#             X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#             X_batch = torch.transpose(torch.transpose(X_batch, 1, 2), 2, 3)
#             y_pred = model(X_batch)
#             loss = criterion(y_pred, y_batch)
#             val_loss += loss.data

#     train_loss = train_loss / len(train_loader)
#     val_loss = val_loss / len(val_loader)
    
#     writer.add_scalars("losses_step", {"val_loss": val_loss}, (epoch + 1) * len(train_loader) - 1)
    
#     writer.add_scalars("losses_epoch", {"train_loss": train_loss}, epoch)
#     writer.add_scalars("losses_epoch", {"val_loss": val_loss}, epoch)
    
    
#     print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
#         epoch+1, 
#         train_loss,
#         val_loss
#     ))
    
#     checkpoint = {
#         'epoch': epoch + 1,
#         'valid_loss_min': val_loss,
#         'state_dict': model.state_dict(),
#         'optimizer': optimizer.state_dict(),
#     }
    
#     torch.save(checkpoint, f'checkpoints/{model_name}/checkpoint_{now}.pt')
#     if val_loss <= val_loss_min:
#         print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_min,val_loss))
#         torch.save(checkpoint, f'checkpoints/{model_name}/best_checkpoint_{now}.pt')
#         val_loss_min = val_loss

In [5]:
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda:0" if use_cuda else "cpu")aset
# model.to(device)
# # checkpoint = torch.load(f'checkpoints/{model_name}/best_checkpoint.pt')
# # model.load_state_dict(checkpoint['state_dict'])
# model.eval()
# m = nn.Softmax(dim=1)

# dset.set_mode('test')
# test_loader = DataLoader(
#     dset, batch_size=100, shuffle=False, collate_fn=pad_collate
# )

# # m = nn.Sigmoid()
# y_pred_extended = []
# y_true_extended = []
# with torch.no_grad():
#     for batch_idx, (X_batch, y_batch) in enumerate(tqdm(test_loader, desc=f"Inference")):
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         X_batch = torch.transpose(torch.transpose(X_batch, 1, 2), 2, 3)
#         y_pred = model(X_batch).cpu()
#         y_pred = m(y_pred)
#         y_pred = np.argmax(y_pred, axis=1)
#         y_pred_extended.extend(y_pred)
#         y_true_extended.extend(y_batch.cpu())
        
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         #print("Normalized confusion matrix")
#     #else:
#         #print('Confusion matrix, without normalization')

#     #print(cm)
    
#     plt.figure(figsize=(25,25))
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, format(cm[i, j], fmt),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.tight_layout()

# # single label
# accuracy = accuracy_score(y_pred_extended, y_true_extended)
# cr = classification_report(y_true_extended, y_pred_extended)
# print(accuracy)
# print(cr)
# cnf_matrix = confusion_matrix(y_true_extended, y_pred_extended)
# # Plot non-normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=[str(i) for i in list(range(10))], title = ('Confusion Matrix'))
# plt.show()

# # # multilabel
# # accuracy = multilabel_acc(y_pred_extended, y_true_extended)
# # # precision = multilabel_prec(y_pred_extended, y_true_extended)
# # # recall = multilabel_recall(y_pred_extended, y_true_extended)
# # print(accuracy)  # , precision, recall)